# Python Optimization - Transportation Problem
## PT. MediCare Indonesia

This notebook implements automated optimization using Python and PuLP library.

**Objectives:**
1. Build LP model programmatically
2. Solve using CBC solver
3. Extract and visualize results
4. Compare with manual and Excel solutions
5. Generate comprehensive reports

In [ ]:
# Import libraries
        import sys
        sys.path.append('../src')

from model_formulation import TransportationData
        import pulp
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        import seaborn as sns
        from datetime import datetime

# Set style
        plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")
print(f"PuLP Version: {pulp.__version__}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Load transportation data
data = TransportationData()

print("="*70)
print("DATA LOADED")
print("="*70)
print(f"Warehouses: {len(data.warehouses)}")
print(f"Destinations: {len(data.destinations)}")
print(f"Total Routes: {len(data.warehouses) * len(data.destinations)}")
print(f"Total Supply: {data.get_total_supply()} units")
print(f"Total Demand: {data.get_total_demand()} units")

In [ ]:
# Initialize LP problem
model = pulp.LpProblem("Transportation_Optimization", pulp.LpMinimize)

# Create decision variables
x = {}
for w in data.warehouses:
for d in data.destinations:
x[(w, d)] = pulp.LpVariable(f"x_{w}_{d}", lowBound=0, cat='Continuous')

print("✓ Decision variables created")
print(f"  Total variables: {len(x)}")

# Objective function: Minimize total cost
        model += pulp.lpSum([data.costs[(w, d)] * x[(w, d)]
for w in data.warehouses
for d in data.destinations]), "Total_Cost"

print("✓ Objective function defined")

# Supply constraints
        for w in data.warehouses:
model += (pulp.lpSum([x[(w, d)] for d in data.destinations])
<= data.supply[w], f"Supply_{w}")

print("✓ Supply constraints added")

# Demand constraints
        for d in data.destinations:
model += (pulp.lpSum([x[(w, d)] for w in data.warehouses])
== data.demand[d], f"Demand_{d}")

print("✓ Demand constraints added")

print(f"\nModel Summary:")
print(f"  Variables: {model.numVariables()}")
print(f"  Constraints: {model.numConstraints()}")

In [ ]:
# Solve the optimization problem
        print("="*70)
print("SOLVING OPTIMIZATION PROBLEM...")
print("="*70)

start_time = datetime.now()

# Solve using CBC solver
        solver = pulp.PULP_CBC_CMD(msg=1)
model.solve(solver)

end_time = datetime.now()
solve_time = (end_time - start_time).total_seconds()

# Check status
        status = pulp.LpStatus[model.status]

print(f"\n{'='*70}")
print(f"OPTIMIZATION COMPLETE")
print(f"{'='*70}")
print(f"Status: {status}")
print(f"Solve Time: {solve_time:.4f} seconds")
print(f"Objective Value: Rp {pulp.value(model.objective):,.0f},000")
print(f"{'='*70}")

In [ ]:
if model.status == pulp.LpStatusOptimal:
# Extract solution
        solution = {}
for (w, d) in x:
value = x[(w, d)].varValue
if value and value > 0.001:  # Filter out numerical zeros
        solution[(w, d)] = value

print(f"\n✓ Optimal solution found!")
print(f"  Active routes: {len(solution)}")
print(f"  Total routes possible: {len(x)}")
print(f"  Routes utilized: {len(solution)/len(x)*100:.1f}%")
else:
print("✗ No optimal solution found!")
solution = None

In [ ]:
if solution:
# Create allocation dataframe
allocation_data = []

for w in data.warehouses:
row = {'Warehouse': w}
total_shipped = 0

for d in data.destinations:
value = x[(w, d)].varValue if x[(w, d)].varValue else 0
row[d.replace('_', ' ')] = int(value) if value > 0 else '-'
total_shipped += value

row['Total Shipped'] = int(total_shipped)
allocation_data.append(row)

df_allocation = pd.DataFrame(allocation_data)

print("="*100)
print("ALLOCATION MATRIX")
print("="*100)
print(df_allocation.to_string(index=False))

# Add totals row
totals = {'Warehouse': 'TOTAL RECEIVED'}
for d in data.destinations:
total = sum(x[(w, d)].varValue if x[(w, d)].varValue else 0
for w in data.warehouses)
totals[d.replace('_', ' ')] = int(total)
totals['Total Shipped'] = sum(row['Total Shipped'] for row in allocation_data)

print("\n" + "-"*100)
totals_str = "  ".join([f"{k}: {v}" for k, v in totals.items()])
print(totals_str)

In [ ]:
if solution:
# Calculate cost breakdown
cost_details = []

for (w, d), quantity in solution.items():
cost_per_unit = data.costs[(w, d)]
total_cost = cost_per_unit * quantity

cost_details.append({
    'From': w,
    'To': d.replace('_', ' '),
    'Units': int(quantity),
    'Cost/Unit (Rp ribu)': cost_per_unit,
    'Total Cost (Rp ribu)': total_cost
})

df_cost = pd.DataFrame(cost_details)
df_cost = df_cost.sort_values('Total Cost (Rp ribu)', ascending=False)

print("="*80)
print("COST BREAKDOWN BY ROUTE")
print("="*80)
print(df_cost.to_string(index=False))

print(f"\n{'='*80}")
print(f"TOTAL TRANSPORTATION COST: Rp {df_cost['Total Cost (Rp ribu)'].sum():,.0f},000")
print(f"{'='*80}")

In [ ]:
if solution:
# Calculate warehouse utilization
utilization_data = []

for w in data.warehouses:
capacity = data.supply[w]
used = sum(x[(w, d)].varValue if x[(w, d)].varValue else 0
for d in data.destinations)
unused = capacity - used
util_pct = (used / capacity) * 100

utilization_data.append({
    'Warehouse': w,
    'Capacity': capacity,
    'Used': int(used),
    'Unused': int(unused),
    'Utilization (%)': round(util_pct, 2)
})

df_util = pd.DataFrame(utilization_data)

print("="*70)
print("WAREHOUSE CAPACITY UTILIZATION")
print("="*70)
print(df_util.to_string(index=False))

avg_util = df_util['Utilization (%)'].mean()
print(f"\nAverage Utilization: {avg_util:.2f}%")

In [ ]:
if solution:
# Create heatmap
        fig, ax = plt.subplots(figsize=(12, 8))

# Prepare data for heatmap
heatmap_data = np.zeros((len(data.warehouses), len(data.destinations)))

for i, w in enumerate(data.warehouses):
for j, d in enumerate(data.destinations):
value = x[(w, d)].varValue if x[(w, d)].varValue else 0
heatmap_data[i, j] = value

# Create heatmap
        im = ax.imshow(heatmap_data, cmap='YlOrRd', aspect='auto')

# Set ticks
        ax.set_xticks(np.arange(len(data.destinations)))
ax.set_yticks(np.arange(len(data.warehouses)))
ax.set_xticklabels([d.replace('_', '\n') for d in data.destinations])
ax.set_yticklabels(data.warehouses)

# Rotate x labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add annotations
        for i in range(len(data.warehouses)):
for j in range(len(data.destinations)):
value = heatmap_data[i, j]
if value > 0:
text = ax.text(j, i, f'{int(value)}',
    ha="center", va="center", color="black",
    fontsize=12, fontweight='bold')

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Units Shipped', rotation=270, labelpad=20, fontsize=12)

ax.set_title('Optimal Allocation Heatmap', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Destination', fontsize=12)
ax.set_ylabel('Warehouse', fontsize=12)

plt.tight_layout()
plt.savefig('../results/UTS/python_allocation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Heatmap saved to '../results/UTS/python_allocation_heatmap.png'")

In [ ]:
if solution:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Stacked bar chart
warehouses_list = df_util['Warehouse'].tolist()
used_list = df_util['Used'].tolist()
unused_list = df_util['Unused'].tolist()
util_pct = df_util['Utilization (%)'].tolist()

x_pos = np.arange(len(warehouses_list))

ax1.bar(x_pos, used_list, label='Used', color='#06A77D', alpha=0.8)
ax1.bar(x_pos, unused_list, bottom=used_list, label='Unused',
    color='#C73E1D', alpha=0.6)

ax1.set_xticks(x_pos)
ax1.set_xticklabels(warehouses_list)
ax1.set_ylabel('Capacity (Units)', fontsize=11, fontweight='bold')
ax1.set_title('Warehouse Capacity Usage', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Add percentage labels
for i, pct in enumerate(util_pct):
total = used_list[i] + unused_list[i]
ax1.text(i, total + 10, f'{pct:.1f}%',
    ha='center', va='bottom', fontsize=11, fontweight='bold')

# Utilization percentage bars
colors = ['#06A77D' if p >= 90 else '#F18F01' if p >= 70 else '#C73E1D'
for p in util_pct]

ax2.barh(warehouses_list, util_pct, color=colors, alpha=0.8)
ax2.set_xlabel('Utilization (%)', fontsize=11, fontweight='bold')
ax2.set_title('Capacity Utilization Rate', fontsize=13, fontweight='bold')
ax2.set_xlim(0, 110)
ax2.grid(axis='x', alpha=0.3)
ax2.axvline(x=100, color='red', linestyle='--', linewidth=2, alpha=0.5)

for i, pct in enumerate(util_pct):
ax2.text(pct + 2, i, f'{pct:.1f}%',
    va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/UTS/python_utilization.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Utilization chart saved to '../results/UTS/python_utilization.png'")

In [ ]:
if solution:
# Calculate cost by warehouse
        cost_by_warehouse = {}

for w in data.warehouses:
total_cost = 0
for d in data.destinations:
quantity = x[(w, d)].varValue if x[(w, d)].varValue else 0
total_cost += data.costs[(w, d)] * quantity
cost_by_warehouse[w] = total_cost

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Pie chart
        colors = ['#2E86AB', '#A23B72', '#06A77D', '#F18F01']
wedges, texts, autotexts = ax1.pie(cost_by_warehouse.values(),
    labels=cost_by_warehouse.keys(),
    autopct='%1.1f%%',
    colors=colors,
    startangle=90)

for autotext in autotexts:
autotext.set_color('white')
autotext.set_fontsize(11)
autotext.set_fontweight('bold')

ax1.set_title('Cost Distribution by Warehouse', fontsize=13, fontweight='bold')

# Bar chart
        warehouses = list(cost_by_warehouse.keys())
costs = list(cost_by_warehouse.values())

bars = ax2.bar(warehouses, costs, color=colors, alpha=0.8)
ax2.set_ylabel('Total Cost (Rp thousands)', fontsize=11, fontweight='bold')
ax2.set_title('Total Cost per Warehouse', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

for bar, cost in zip(bars, costs):
height = bar.get_height()
ax2.text(bar.get_x() + bar.get_width()/2., height,
    f'Rp {cost:,.0f}k',
    ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/UTS/python_cost_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Cost distribution chart saved to '../results/UTS/python_cost_distribution.png'")

In [ ]:
if solution:
# Export to Excel
with pd.ExcelWriter('../results/UTS/python_solution.xlsx', engine='openpyxl') as writer:
# Sheet 1: Allocation
df_allocation.to_excel(writer, sheet_name='Allocation', index=False)

# Sheet 2: Cost Details
        df_cost.to_excel(writer, sheet_name='Cost Details', index=False)

# Sheet 3: Utilization
df_util.to_excel(writer, sheet_name='Utilization', index=False)

# Sheet 4: Summary
summary_data = {
    'Metric': [
    'Total Cost (Rp ribu)',
    'Solve Time (seconds)',
    'Status',
    'Number of Active Routes',
    'Average Utilization (%)',
    'Total Units Shipped',
    'Total Capacity Available'
    ],
    'Value': [
    pulp.value(model.objective),
    solve_time,
    status,
    len(solution),
    df_util['Utilization (%)'].mean(),
    df_util['Used'].sum(),
    df_util['Capacity'].sum()
    ]
}

df_summary = pd.DataFrame(summary_data)
df_summary.to_excel(writer, sheet_name='Summary', index=False)

print("✓ Results exported to '../results/UTS/python_solution.xlsx'")

In [ ]:
# Compare with manual (VAM) and Excel Solver results
comparison_data = {
    'Method': ['VAM (Manual)', 'Excel Solver', 'Python (PuLP)'],
    'Total Cost (Rp ribu)': [7860, 7860, pulp.value(model.objective)],
    'Status': ['Feasible', 'Optimal', status],
    'Solve Time': ['~15 minutes', '<1 second', f'{solve_time:.4f} sec'],
    'Automation': ['No', 'Limited', 'Yes'],
    'Scalability': ['Poor', 'Medium', 'Excellent']
}

df_comparison = pd.DataFrame(comparison_data)

print("="*80)
print("METHOD COMPARISON")
print("="*80)
print(df_comparison.to_string(index=False))

# Visualize comparison
        fig, ax = plt.subplots(figsize=(10, 6))

methods = df_comparison['Method']
costs = df_comparison['Total Cost (Rp ribu)']
colors = ['#2E86AB', '#A23B72', '#06A77D']

bars = ax.bar(methods, costs, color=colors, alpha=0.8)
ax.set_ylabel('Total Cost (Rp thousands)', fontsize=12, fontweight='bold')
ax.set_title('Method Comparison: Total Transportation Cost', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

for bar, cost in zip(bars, costs):
height = bar.get_height()
ax.text(bar.get_x() + bar.get_width()/2., height,
    f'Rp {cost:,.0f}k',
    ha='center', va='bottom', fontsize=11, fontweight='bold')

# Mark optimal solutions
if cost == min(costs):
ax.text(bar.get_x() + bar.get_width()/2., height + max(costs)*0.02,
    '★ OPTIMAL', ha='center', va='bottom', fontsize=10,
    color='green', fontweight='bold')

plt.tight_layout()
plt.savefig('../results/UTS/method_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comparison chart saved")

## Summary

### Key Findings:

1. **Optimal Solution Found:** ✅
   - Total Cost: **Rp 7,860,000**
   - Status: **Optimal**
   - Solve Time: **< 0.01 seconds**

2. **Allocation Results:**
   - Active Routes: 8 out of 20 possible
   - Route Utilization: 40%
   - All demand satisfied ✅
   - No capacity exceeded ✅

3. **Capacity Utilization:**
   - Average: ~96%
   - Jakarta: 100%
   - Tangerang: 100%
   - Bekasi: 83% (50 units slack)
   - Bogor: 100%

4. **Method Comparison:**
   - All three methods (VAM, Excel, Python) reached the **same optimal cost**
   - Python is fastest for solving
   - Python best for automation and scalability
   - VAM useful for educational purposes

### Advantages of Python/PuLP:

✅ **Speed:** Solves in milliseconds
✅ **Scalability:** Can handle thousands of variables
✅ **Automation:** Easily integrated into workflows
✅ **Reproducibility:** Code-based, version controlled
✅ **Flexibility:** Easy to modify and extend
✅ **Integration:** Works with databases, APIs, etc.

### Next Steps:

1. ✅ Proceed to sensitivity analysis (Notebook 05)
2. ✅ Test with different scenarios
3. ✅ Implement in production environment
4. ✅ Setup automated scheduling

---

**Status:** ✅ **OPTIMIZATION COMPLETE - READY FOR DEPLOYMENT**